In [17]:
from collections import Counter
import random


class Board:
    def __init__(self):
        self.board = [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']

    def __str__(self):
        return("\n 0 | 1 | 2     %s | %s | %s\n"
               "---+---+---   ---+---+---\n"
               " 3 | 4 | 5     %s | %s | %s\n"
               "---+---+---   ---+---+---\n"
               " 6 | 7 | 8     %s | %s | %s" % (self.board[0], self.board[1], self.board[2],
                                                self.board[3], self.board[4], self.board[5],
                                                self.board[6], self.board[7], self.board[8]))

    def valid_move(self, move):
        try:
            move = int(move)
        except ValueError:
            return False
        if 0 <= move <= 8 and self.board[move] == ' ':
            return True
        return False

    def winning(self):
        return ((self.board[0] != ' ' and
                 ((self.board[0] == self.board[1] == self.board[2]) or
                  (self.board[0] == self.board[3] == self.board[6]) or
                  (self.board[0] == self.board[4] == self.board[8])))
                or (self.board[4] != ' ' and
                    ((self.board[1] == self.board[4] == self.board[7]) or
                    (self.board[3] == self.board[4] == self.board[5]) or
                    (self.board[2] == self.board[4] == self.board[6])))
                or (self.board[8] != ' ' and
                    ((self.board[2] == self.board[5] == self.board[8]) or
                    (self.board[6] == self.board[7] == self.board[8]))))

    def draw(self):
        return all((x != ' ' for x in self.board))

    def play_move(self, position, marker):
        self.board[position] = marker

    def board_string(self):
        return ''.join(self.board)


class MenacePlayer:
    def __init__(self):
        self.matchboxes = {}
        self.num_win = 0
        self.num_draw = 0
        self.num_lose = 0

    def start_game(self):
        self.moves_played = []

    def get_move(self, board):
        # Find board in matchboxes and choose a bead
        # If the matchbox is empty, return -1 (resign)
        board = board.board_string()
        if board not in self.matchboxes:
            new_beads = [pos for pos, mark in enumerate(board) if mark == ' ']
            # Early boards start with more beads
            self.matchboxes[board] = new_beads * ((len(new_beads) + 2) // 2)

        beads = self.matchboxes[board]
        if len(beads):
            bead = random.choice(beads)
            self.moves_played.append((board, bead))
        else:
            bead = -1
        return bead

    def win_game(self):
        # We won, add three beads
        for (board, bead) in self.moves_played:
            self.matchboxes[board].extend([bead, bead, bead])
        self.num_win += 1

    def draw_game(self):
        # A draw, add one bead
        for (board, bead) in self.moves_played:
            self.matchboxes[board].append(bead)
        self.num_draw += 1

    def lose_game(self):
        # Lose, remove a bead
        for (board, bead) in self.moves_played:
            matchbox = self.matchboxes[board]
            del matchbox[matchbox.index(bead)]
        self.num_lose += 1

    def print_stats(self):
        print('Have learnt %d boards' % len(self.matchboxes))
        print('W/D/L: %d/%d/%d' % (self.num_win, self.num_draw, self.num_lose))

    def print_probability(self, board):
        board = board.board_string()
        try:
            print("Stats for this board: " +
                  str(Counter(self.matchboxes[board]).most_common()))
        except KeyError:
            print("Never seen this board before.")


class HumanPlayer:
    def __init__(self):
        pass

    def start_game(self):
        print("Get ready!")

    def get_move(self, board):
        while True:
            move = input('Make a move: ')
            if board.valid_move(move):
                break
            print("Not a valid move")
        return int(move)

    def win_game(self):
        print("You won!")

    def draw_game(self):
        print("It's a draw.")

    def lose_game(self):
        print("You lose.")

    def print_probability(self, board):
        pass


def play_game(first, second, silent=False):
    first.start_game()
    second.start_game()
    board = Board()

    if not silent:
        print("\n\nStarting a new game!")
        print(board)

    while True:
        if not silent:
            first.print_probability(board)
        move = first.get_move(board)
        if move == -1:
            if not silent:
                print("Player resigns")
            first.lose_game()
            second.win_game()
            break
        board.play_move(move, 'X')
        if not silent:
            print(board)
        if board.winning():
            first.win_game()
            second.lose_game()
            break
        if board.draw():
            first.draw_game()
            second.draw_game()
            break

        if not silent:
            second.print_probability(board)
        move = second.get_move(board)
        if move == -1:
            if not silent:
                print("Player resigns")
            second.lose_game()
            first.win_game()
            break
        board.play_move(move, 'O')
        if not silent:
            print(board)
        if board.winning():
            second.win_game()
            first.lose_game()
            break


if __name__ == '__main__':
    go_first_menace = MenacePlayer()
    go_second_menace = MenacePlayer()
    human = HumanPlayer()

    for i in range(1000):
        play_game(go_first_menace, go_second_menace, silent=True)

    go_first_menace.print_stats()
    go_second_menace.print_stats()

    play_game(go_first_menace, human)
    play_game(human, go_second_menace)


Have learnt 1289 boards
W/D/L: 570/136/294
Have learnt 1127 boards
W/D/L: 294/136/570
Get ready!


Starting a new game!

 0 | 1 | 2       |   |  
---+---+---   ---+---+---
 3 | 4 | 5       |   |  
---+---+---   ---+---+---
 6 | 7 | 8       |   |  
Stats for this board: [(8, 324), (6, 296), (7, 288), (4, 220), (0, 139), (5, 131), (1, 116), (2, 49), (3, 34)]

 0 | 1 | 2       |   |  
---+---+---   ---+---+---
 3 | 4 | 5       | X |  
---+---+---   ---+---+---
 6 | 7 | 8       |   |  
Make a move: 0

 0 | 1 | 2     O |   |  
---+---+---   ---+---+---
 3 | 4 | 5       | X |  
---+---+---   ---+---+---
 6 | 7 | 8       |   |  
Stats for this board: [(1, 13), (2, 13), (5, 10), (8, 8), (7, 4), (3, 3), (6, 3)]

 0 | 1 | 2     O |   |  
---+---+---   ---+---+---
 3 | 4 | 5     X | X |  
---+---+---   ---+---+---
 6 | 7 | 8       |   |  
Make a move: 5

 0 | 1 | 2     O |   |  
---+---+---   ---+---+---
 3 | 4 | 5     X | X | O
---+---+---   ---+---+---
 6 | 7 | 8       |   |  
Never seen this b

KeyboardInterrupt: ignored